# Metaheurísticas

## Representação de uma solução viável (ideia inicial)

- Cada sonda guarda uma lista (array) de projetos executados por ela (map <sonda, lista>)

    - guardar projeto e tempos de início e término

- Manter um contador de gastos (geral e por tipo de projeto)

- Cada sonda guarda uma lista de janelas de tempo disponíveis

- Cada sonda guarda uma lista de possíveis projetos (heap, com key por critério guloso)

## Representação de uma solução viável (segunda versão)

- Cada sonda quarda uma lista duplamente encadeada de atividades

    - Atividades podem ser janelas de tempo ou alocações de projetos
    
    - Cada item da lista contém (atividade, prev, inicio, final)

- Manter um contador de gastos (geral e por tipo de projeto)

- Cada sonda guarda uma lista de possíveis projetos (heap, com key por critério guloso)

## Estruturas de dados

- Linked list:

- Heap:

# Critérios gulosos

- lucro

- lucro / custo

- lucro / duração

- lucro / distância

## Heurística de construção

1. Escolher uma sonda
2. Escolher um projeto (dentro da heap de possíveis projetos)
3. Alocar na sonda
    - se for inviável (pelo tempo), tentar realocar até k projetos (menos restritivos dentro da janela do projeto a ser inserido)
    - ou tentar realocar por sliding window
4. Atualizar dados (gastos, janelas de tempo, projetos possíveis)
5. Repetir até restrições serem alcançadas


## Heurística de refinamento (busca local)

### Estrutura de vizinhança 1: Inserir projeto ainda não escolhido

- Escolher projeto por critério guloso (dentro dos não selecionados)

- Tentar inserir sem remover ninguém

- Se for inviável, então, dentro da janela do projeto a ser inserido:

    - percorrer todos os projetos e escolher o maior ganho / menor perda, ao inserir o novo projeto (sliding window)
    
    - ou tentar realocar até k projetos (os k menos restritivos dentro da janela do projeto a ser inserido)

### Estrutura de vizinhança 2: Remover k projetos e preencher com heurística de construção

- Posso variar k projetos a serem removidos

- Posso variar critério de remoção e reconstrução para maior diversificação


## Crossover

### Ideia 1: quando a instância tem mais de uma sonda

Dadas duas soluções possíveis:

- Dividir as sondas em dois conjuntos

- Para a primeira solução, escolher o primeiro conjunto de sondas (solução parcial)

- Para a segunda solução, escolher o segundo conjunto de sondas (solução complementar)

- Remover aleatoriamente projetos que estejam tanto na solução parcial, quanto na complementar

- Unir solução parcial com solução complementar

- Para cada sonda

    - Preencher buracos com heurística de construção

### Idéia 2: mais genérica

Dadas duas soluções possíveis, para cada sonda:

- Dividir os projetos em dois grupos

- Para a primeira solução, escolher o primeiro conjunto de projetos (solução parcial)

- Para a segunda solução, escolher o segundo conjunto de projetos (solução complementar)

- Remover aleatoriamente projetos que estejam tanto na solução parcial, quanto na complementar

- Para cada projeto da solução complementar:

    - Tentar inserir na solução parcial usando heurística de construção (projetos já selecionados)
    
- Preencher buracos com heurística de construção

## Decodificador BRKGA

### Representação:

- Cada projeto recebe um número aleatório entre 0 e 1

### Ideia:

- Usar a heurística de construção, simplificada:

    - Critério guloso: ordem dos números aleatórios
    
    - Eficiência: não pegar pesado em tentar viabilizar a inserção de projetos inviáveis

- Construir parte da população inicial com GRASP

## Lendo dados

In [ ]:
import copy

In [ ]:
ls

In [ ]:
coords_x = []
coords_y = []
bacias = []
nomes = []
maturidades = []
qualidades = []
plays = []
soterramentos = []
pcgnas = []
geracao = []
migracao = []
reservatorio = []
geometria = []
retencao = []
pshc = []
mc_vol = []#print ('sonda: ', sonda, ' inserindo projeto ', projeto, ' nos tempos ', release, ' e ', 
                
mi_vol = []
mc_vpl = []
mi_vpl = []
custos = []
tempos_exec = []
inicio_janela = []
final_janela = []
sondas_x = []
sondas_y = []

data = {
    1: coords_x,
    2: coords_y,
    3: bacias,
    4: nomes,
    5: maturidades,
    6: qualidades,
    7: plays,
    8: soterramentos,
    9: pcgnas,
    10: geracao,
    11: migracao,
    12: reservatorio,
    13: geometria,
    14: retencao,
    15: pshc,
    16: mc_vol,
    17: mi_vol,
    18: mc_vpl,
    19: mi_vpl,
    20: custos,
    21: tempos_exec,
    22: inicio_janela,
    23: final_janela,
    25: sondas_x,
    26: sondas_y
}

with open('instancia_10projetos_2sondas.dat', 'r') as f:
    for i, line in enumerate(f):
        if i == 0:
            (n_projetos, custo_total) = int(line.split()[0]), float(line.split()[1])
        elif i == 24:
            n_sondas = int(line.split()[0])
        else:
            for elem in line.split('[')[1].split(']')[0].split(','):
                try:
                    data[i].append(float(elem))
                except:
                    data[i].append(elem)

In [ ]:
n_projetos

In [ ]:
custo_total

In [ ]:
n_sondas

In [ ]:
def ConstruirSolucao(dados, n_projetos, n_sondas, custo_total, t_init, t_final, delta_t, criterio='lucro/custo'):
    
    import math
    import copy
    import numpy as np
    import heapq
    
    dados_local = copy.deepcopy(dados)
    
    n_periodos = ((t_final - t_init) // delta_t) + 1
    
    # convertendo dados de tempo para períodos -------------- TODO: verificar se não está somando 1 a mais
    # ------------------------------------------------------- TODO: posso fazer isso como pré-processamento
    for i in range(n_projetos):
        dados_local[21][i] = (dados_local[21][i] // delta_t) + 1
        dados_local[22][i] = (dados_local[22][i] // delta_t)
        dados_local[23][i] = (dados_local[23][i] // delta_t)
    
    # calculando tempo de deslocamento entre projetos: ---------- TODO: posso fazer isso como pré-processamento
    desloc = []
    lag = n_sondas
    for i in range(n_projetos+n_sondas):
        desloc.append([])
        for j in range(n_projetos+n_sondas):
            if (i<n_sondas) and (j<n_sondas):
                dist = math.sqrt( (dados[25][i] - dados[25][j])**2 + (dados[26][i] - dados[26][j])**2 )
            elif (i<n_sondas) and not(j<n_sondas):
                dist = math.sqrt( (dados[25][i] - dados[1][j-lag])**2 + (dados[25][i] - dados[2][j-lag])**2 )
            elif not(i<n_sondas) and (j<n_sondas):
                dist = math.sqrt( (dados[1][i-lag] - dados[26][j])**2 + (dados[2][i-lag] - dados[26][j])**2 )
            else:
                dist = math.sqrt( (dados[1][i-lag] - dados[1][j-lag])**2 + (dados[2][i-lag] - dados[2][j-lag])**2 )
            
            # converte para períodos: --------------------- TODO : verificar se não está somando 1 a mais
            if delta_t == 1:
                desloc[i].append(dist)
            else:
                desloc[i].append((dist // delta_t) + 1)
    
    # inicializando solução
    s = {i:[] for i in range(n_sondas)}
    
    # inicializando janelas de tempo, por sonda
    s_janelas = {i:[[-1, [0, n_periodos]]] for i in range(n_sondas)}
    
    # inicializando lista de candidatos, por sonda
    s_candidatos = {i:[] for i in range(n_sondas)}
    
    # armazena lista de candidatos como uma heap: ------- OBS: valores keys multiplicados por -1 (MaxHeap)
    # ---------------------------------- TODO: posso ordenar projetos pelo critério guloso como pré-processamento?
    for i in range(n_sondas):
        for j in range(n_projetos):
            criterio_val = dados_local[19][j] / dados_local[20][j]
            heapq.heappush( s_candidatos[i], (-criterio_val, j) )
    
    # inicializa contador de gastos total
    gastos = 0
    
    proj_usados = set()
    sondas = set([i for i in range(n_sondas)])
    
    fitness = 0
    
    while (gastos <= custo_total and sondas):
        
        # escolher uma sonda
        sonda = np.random.choice(list(sondas))
        
        # escolher um projeto
        projeto = heapq.heappop(s_candidatos[sonda])[1]
        
        # se sonda não tem mais candidatos, remover ela da lista
        if s_candidatos[sonda] == []:
            sondas.remove(sonda)
        
        # se projeto já foi escolhido, pular
        if (projeto in proj_usados):
            continue
        
        # se existe janela disponível na sonda
        aloc = False
        proc = dados_local[21][projeto]
        for i, (prev, janela) in enumerate(s_janelas[sonda]):
            release = max(janela[0], dados_local[22][projeto])
            due = min(janela[1], dados_local[23][projeto])
            if prev == -1:
                setup = desloc[sonda][projeto + lag]
            else:
                setup = desloc[prev + lag][projeto + lag]
            if ( release + proc + setup <= due ):
                
                #print ('sonda: ', sonda, ' inserindo projeto ', projeto, ' nos tempos ', release, ' e ', 
                #       release+setup+proc, ' dentro da janela ', janela[0], janela[1])
                
                # alocar
                s[sonda].append((copy.copy(projeto), copy.copy(release), copy.copy(release+setup+proc)))
                proj_usados.add(copy.copy(projeto))
                fitness += dados_local[19][projeto]
                aloc = True
                
                # atualizar janela da sonda
                
                if ( (int(janela[0])==int(release)) and (int(janela[1])==int(release+setup+proc)) ):
                    # remover janela
                    s_janelas[sonda].pop(i)
                    
                    #https://www.youtube.com/watch?v=3pUdUX5Gagcprint ('janela toda preenchida pelo projeto')
                    
                elif (int(janela[0])==int(release)):
                    # atualizar release da janela e colocar o prev como o projeto alocado
                    s_janelas[sonda][i][0] = copy.copy(projeto)
                    s_janelas[sonda][i][1][0] = copy.copy(release + setup + proc)
                    
                    #print ('projeto preenche inicio da janela')
                    
                elif (int(janela[1])==int(release+setup+proc)):
                    # atualiza due da janela, mantendo o prev
                    s_janelas[sonda][i][1][1] = copy.copy(release)
                    
                    #print ('projeto preenche final da janela')
                    
                else:
                    # abrir janela antes da release, cujo prev é o prev original
                    s_janelas[sonda].insert(i, [copy.copy(prev), [copy.copy(janela[0]), copy.copy(release)]])
                    # e atualizar prev e release da janela original
                    s_janelas[sonda][i+1][0] = copy.copy(projeto)
                    s_janelas[sonda][i+1][1][0] = copy.copy(release + setup + proc)
                    
                    #print ('projeto preenche meio da janela')
                    
                break
            else:
                # senão, tentar realocar projetos ---------------------------------- TODO
                pass
        
        # atualizar dados de gastos
        if (aloc):
            gastos += dados_local[20][projeto]
    
    return fitness, s, s_janelas, s_candidatos

In [ ]:
t_init, t_final, delta_t = 0, 5*12*4*7, 7

In [ ]:
ConstruirSolucao(data, n_projetos, n_sondas, custo_total, t_init, t_final, delta_t)

In [ ]:
def BuscaLocalSoft():
    
    return

In [ ]:
def BuscarLocalHard():
    
    return

In [ ]:
def GRASP():
    
    return